In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adilguptagwl","key":"8d9b84d8f53905aa1d70492366b4f10f"}'}

In [2]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

rm: cannot remove '/root/.kaggle': No such file or directory
ref                                                       title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ruchi798/data-science-job-salaries                        Data Science Job Salaries                            7KB  2022-06-15 08:59:12           6773        226  1.0              
surajjha101/bigbasket-entire-product-list-28k-datapoints  BigBasket Entire Product List (~28K datapoints)      6MB  2022-06-22 12:51:18           2244         97  1.0              
victorsoeiro/netflix-tv-shows-and-movies                  Netflix TV Shows and Movies                          2MB  2022-05-15 00:01:23          17011        492  1.0              
sameepvani/nasa-nearest-earth-obje

In [3]:
!kaggle datasets download -d brycecf/give-me-some-credit-dataset

  0% 0.00/5.16M [00:00<?, ?B/s]
100% 5.16M/5.16M [00:00<00:00, 168MB/s]


In [5]:
!unzip give-me-some-credit-dataset.zip

Archive:  give-me-some-credit-dataset.zip
replace Data Dictionary.xls? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
#Read the DATA
import pandas as pd
df_train = pd.read_csv('cs-training.csv')

df_train.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [7]:
#shape of data
print(df_train.shape)

(150000, 12)


In [8]:
#check  dataset imbalance
df_train['SeriousDlqin2yrs'].sum()
#stratified k fold is necessary

10026

In [9]:
#minimal prepprocessing
cols_missing = [col for col in df_train.columns if df_train[col].isnull().any()]

df_train = df_train.drop(cols_missing,axis =1)

for col in df_train.columns:
    df_train[col] = df_train[col].astype(float)

In [10]:
#import possibly good models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

lr = LogisticRegression()
gnb = GaussianNB()
rf = RandomForestClassifier()
xgb = XGBClassifier()

In [11]:
#further necessary for baseline model
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

In [12]:

X = df_train.drop(['Unnamed: 0','SeriousDlqin2yrs'],axis =1)
y = df_train['SeriousDlqin2yrs']

In [13]:
#creating stratified k fold subsets
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [14]:
#train , fit , predict , avg auc score
def score(model):
    scoring = ['precision', 'recall', 'f1','roc_auc']
    return cross_validate(model,X,y,cv=folds,scoring = scoring,n_jobs = -1)

In [15]:
models = [lr,gnb,rf,xgb]
scores_dictionary = {}
for model in models:
    scores_dictionary[model] = score(model)

In [16]:
scores_dictionary

{GaussianNB(): {'fit_time': array([0.06553149, 0.0556314 , 0.0536797 , 0.05550718, 0.03644848]),
  'score_time': array([0.10261464, 0.10947967, 0.10660291, 0.10703135, 0.06457758]),
  'test_f1': array([0.04547564, 0.03964134, 0.04356061, 0.05329593, 0.04590164]),
  'test_precision': array([0.32885906, 0.36842105, 0.42990654, 0.42537313, 0.37692308]),
  'test_recall': array([0.02442672, 0.02094763, 0.02294264, 0.02842893, 0.0244389 ]),
  'test_roc_auc': array([0.70407366, 0.72240132, 0.71141584, 0.70730793, 0.71850749])},
 LogisticRegression(): {'fit_time': array([2.62828517, 2.18680334, 2.08196068, 2.2857542 , 1.80329108]),
  'score_time': array([0.08495045, 0.0835557 , 0.0889523 , 0.08584189, 0.0555656 ]),
  'test_f1': array([0.08615949, 0.04484733, 0.06403712, 0.06037736, 0.09845032]),
  'test_precision': array([0.53409091, 0.51648352, 0.46      , 0.55652174, 0.57142857]),
  'test_recall': array([0.04685942, 0.0234414 , 0.03441397, 0.0319202 , 0.05386534]),
  'test_roc_auc': array([0

In [17]:
Baseline_results = pd.DataFrame({'LR':[scores_dictionary[lr]['test_precision'].mean(),scores_dictionary[lr]['test_recall'].mean(),scores_dictionary[lr]['test_f1'].mean(),scores_dictionary[lr]['test_roc_auc'].mean()],
                                'GNB':[scores_dictionary[gnb]['test_precision'].mean(),scores_dictionary[gnb]['test_recall'].mean(),scores_dictionary[gnb]['test_f1'].mean(),scores_dictionary[gnb]['test_roc_auc'].mean()],
                                'RF':[scores_dictionary[rf]['test_precision'].mean(),scores_dictionary[rf]['test_recall'].mean(),scores_dictionary[rf]['test_f1'].mean(),scores_dictionary[rf]['test_roc_auc'].mean()],
                                'XGB':[scores_dictionary[xgb]['test_precision'].mean(),scores_dictionary[xgb]['test_recall'].mean(),scores_dictionary[xgb]['test_f1'].mean(),scores_dictionary[xgb]['test_roc_auc'].mean()],},
                               index = ['precision','recall','F1','ROC AUC'])

In [18]:
Baseline_results

,LR,GNB,RF,XGB
precision,0.527705,0.385897,0.524499,0.597193
recall,0.038100,0.024237,0.193996,0.186217
F1,0.070774,0.045575,0.283196,0.283789
ROC AUC,0.690390,0.712741,0.831798,0.863633


In [20]:
import pickle
pickle.dump(Baseline_results,open("BaselineResults","wb"))
#good job